In [1]:
from mindquantum.utils import random_circuit
from mindquantum.core.circuit import Circuit
from huawei_graph_tool import ConvertCircuit,my_random_circuit

import random,os,torch
import networkx as nx
from mindquantum.io.qasm import HiQASM
from torch_geometric.loader import DataLoader
from torch_geometric.transforms import RandomLinkSplit
# from Pytorch_VGAE import CustomGraphDataset,VGAEModel
import pandas as pd
import torch.nn.functional as F
from torch_geometric.utils import from_scipy_sparse_matrix, dense_to_sparse
from Pytorch_VGAE import CustomGraphDataset,VGAEModel

/opt/miniconda3/envs/MindSpore/lib/python3.9/site-packages/mindquantum/simulator/__init__.py:17: UserWarning: Unable import mqvector gpu backend due to: cannot import name '_mq_vector_gpu' from partially initialized module 'mindquantum' (most likely due to a circular import) (/opt/miniconda3/envs/MindSpore/lib/python3.9/site-packages/mindquantum/__init__.py)
  from .available_simulator import SUPPORTED_SIMULATOR


In [3]:
root = './VGAE_dataset'
dataset = CustomGraphDataset(root=root)

/Users/yangjianfei/mac_vscode/华为 QAS 实习/Pytorch_VGAE.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.data, self.slices = torch.load(self.processed_paths[0])


In [4]:
# 超参数
num_epochs = 200
learning_rate = 0.01
batch_size = 32
out_channels = 16  # 嵌入维度

In [5]:
from torch.optim import Adam
import time
# 创建 DataLoader
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 实例化模型和优化器
model = VGAEModel(in_channels=dataset[0].x.size(1), out_channels=out_channels)
optimizer = Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 训练循环
for epoch in range(num_epochs):
    start_time = time.time()  # 记录开始时间
    model.train()
    total_loss = 0

    for batch_idx, data in enumerate(train_loader):
        optimizer.zero_grad()
        
        # 从图数据中获取节点特征和边索引
        x, edge_index = data.x, data.edge_index
        
        # 前向传播
        z = model.encode(x, edge_index)
        
        # 重建损失
        loss = model.recon_loss(z, edge_index)
        kl_divergence = model.kl_loss()
        total_loss = loss + kl_divergence
        
        # 反向传播
        total_loss.backward()
        optimizer.step()

        # 打印每个批次的损失
        #print(f'  Batch {batch_idx + 1}/{len(train_loader)}, Loss: {total_loss.item():.4f}')

        total_loss += total_loss.item()

    avg_loss = total_loss / len(train_loader)
    elapsed_time = time.time() - start_time  # 计算经过时间
    print(f'Epoch {epoch + 1}/{num_epochs}, Average Loss: {avg_loss:.4f}, Time: {elapsed_time:.2f}s')

In [9]:
# 保存训练后的模型
import pandas as pd
import numpy as np
import scipy.sparse as sp
torch.save(model.state_dict(), 'vgae_model_0930.pth')

# 读取 CSV 文件时不使用表头，确保读取所有数据
A = pd.read_csv('./VGAE_dataset/raw/A/A_2.csv', header=None).values
X = pd.read_csv('./VGAE_dataset/raw/X/X_2.csv', header=None).values
# 检查形状是否正确
print(A.shape)


(32, 32)


In [10]:
# 加载保存的 VGAE 模型权重
model = VGAEModel(in_channels=X.shape[1], out_channels=out_channels)  # 假设 out_channels 是 32
model.load_state_dict(torch.load('vgae_model_0930.pth'))
model.eval()  # 切换到评估模式

/var/folders/8x/k_m4pmb11437ktb_r6tjzt2c0000gn/T/ipykernel_89407/3224179255.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('vgae_model_

VGAEModel(
  (encoder): GINEncoder(
    (conv1): GINConv(nn=Sequential(
      (0): Linear(in_features=22, out_features=32, bias=True)
      (1): ReLU()
      (2): Linear(in_features=32, out_features=16, bias=True)
    ))
    (conv_mu): GINConv(nn=Sequential(
      (0): Linear(in_features=16, out_features=32, bias=True)
      (1): ReLU()
      (2): Linear(in_features=32, out_features=16, bias=True)
    ))
    (conv_logstd): GINConv(nn=Sequential(
      (0): Linear(in_features=16, out_features=32, bias=True)
      (1): ReLU()
      (2): Linear(in_features=32, out_features=16, bias=True)
    ))
  )
  (decoder): InnerProductDecoder()
)

In [12]:
X = torch.tensor(X)
print(X)
print(edge_index)

tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0],
        [0, 

In [16]:
A = sp.coo_matrix(A)  # 如果 A 不是稀疏矩阵，先转换为 COO 格式
edge_index, _ = from_scipy_sparse_matrix(A)

# 进行编码，得到隐向量 z
with torch.no_grad():  # 评估时不需要计算梯度
    z = model.encode(X, edge_index)
    #z.shape = (32, 16)

# 使用解码器重建邻接矩阵
reconstructed_A = model.decoder(z, edge_index)
threshold = 0.5
binary_A = (reconstructed_A > threshold).float()

# 打印编码后的隐变量 z 和重建后的邻接矩阵
print("Encoded latent variable z:")
print(z.shape)

print("Reconstructed adjacency matrix A:")
print(binary_A)

# 可以将 reconstructed_A 处理成具体的矩阵格式展示
reconstructed_A_matrix = torch.sigmoid(torch.matmul(z, z.t())).cpu().numpy()
print("Reconstructed A as a matrix:")
print(reconstructed_A_matrix)

Encoded latent variable z:
torch.Size([32, 16])
Reconstructed adjacency matrix A:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1.])
Reconstructed A as a matrix:
[[0.6117643  0.63777816 0.63888246 ... 0.6749312  0.6614476  0.6231562 ]
 [0.63777816 0.67436415 0.67498016 ... 0.7239567  0.7051634  0.65325826]
 [0.63888246 0.67498016 0.67583776 ... 0.72398627 0.7051954  0.6540061 ]
 ...
 [0.6749312  0.7239567  0.72398627 ... 0.7876996  0.7648372  0.6965454 ]
 [0.6614476  0.7051634  0.7051954  ... 0.7648372  0.74720067 0.6838767 ]
 [0.6231562  0.65325826 0.6540061  ... 0.6965454  0.6838767  0.6398762 ]]
